In [ ]:
import numpy as np
import zipfile as zp
import tensorflow as tf
import tensorflow.keras
import shutil
import os
from PIL import Image
import re
import random
import pickle
import gc
import dill
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Extracing zip files to working dir
train_zp = zp.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip')
train_zp.extractall('/kaggle/working/')

test_zp = zp.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip')
test_zp.extractall('/kaggle/working/')

#Printing number of training and test samples
print('No of training samples  = ' + str(len(os.listdir('/kaggle/working/train'))))
print('No of test samples = ' + str(len(os.listdir('/kaggle/working/test'))))
train_fnames = os.listdir('train/')


In [ ]:
#Creating dir structure for flow from directory API
try:
    os.mkdir('train/cat')
    os.mkdir('train/dog')
    os.mkdir('validation')
    os.mkdir('validation/cat')
    os.mkdir('validation/dog')
except:
    pass

In [ ]:
#Moving training pics to cat/dog subdir based on filename
def move_train_file(path1,path2,x):
   #path specifies either train or test
   #If cat move to cat subdir if not move to dog subdir
    if x[:3] == 'cat':
        shutil.move(path1+'/' +x, path2 + '/cat/'+x)
    elif x[:3] == 'dog':
        shutil.move(path1+'/' +x, path2 + '/dog/'+x)

pd.Series(train_fnames).apply(lambda x: move_train_file('train','train',x))
print(len(os.listdir('train/cat')), len(os.listdir('train/dog')))

In [ ]:
#Randomly pick 20% of training images and move to validation
def move_file(path1,path2,x):
   #path specifies either train or test
   #If cat move to cat subdir if not move to dog subdir
    if x[:3] == 'cat':
        shutil.move(path1+'/cat/' +x, path2 + '/cat/'+x)
    elif x[:3] == 'dog':
        shutil.move(path1 + '/dog/'+x, path2 + '/dog/'+x)
valid_fnames_cat = np.array(os.listdir('train/cat'))[np.random.choice(list(range(12500)), 2500, replace=False)]
valid_fnames_dog = np.array(os.listdir('train/dog'))[np.random.choice(list(range(12500)), 2500, replace=False)]
print(len(valid_fnames_cat), len(valid_fnames_dog))
pd.Series(valid_fnames_cat).apply(lambda x: move_file('train','validation',x))
pd.Series(valid_fnames_dog).apply(lambda x: move_file('train','validation',x))
print('Validation set is ',len(os.listdir('validation/cat')), ' and ',len(os.listdir('validation/dog')))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1/255.0,
                                   #zoom_range = 0.2,
                                   #shear_range = 0.2,
                                   #horizontal_flip = True,
                                    #rotation_range = 45,
                                    #width_shift_range = 0.2,
                                    #height_shift_range = 0.2,
                                   #fill_mode = "nearest"
                                   )
valid_datagen = ImageDataGenerator(rescale = 1/255.0,
                                   #horizontal_flip = True,
                                   #shear_range = 0.2,
                                   #zoom_range = 0.2,
                                   #fill_mode = 'nearest',
                                   #height_shift_range = 0.2,
                                   #width_shift_range = 0.2,
                                   #rotation_range = 40
                                   
)

In [ ]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l2
tensorflow.compat.v1.disable_eager_execution()

In [ ]:
#Specify the model
model = tensorflow.keras.Sequential([
    
    #Conv layer 1
    tf.keras.layers.Conv2D(32, (3, 3), activation='elu',input_shape=(256,256,3), kernel_initializer='he_normal' ),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    #Conv layer 2
    tf.keras.layers.Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    #Conv layer 3
    tf.keras.layers.Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    #Conv layer 4
    tf.keras.layers.Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    #Conv layer 5
    tf.keras.layers.Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    #Flatten
    Flatten(),
    #Fully Connected layer
    Dense(1024, activation='elu', kernel_regularizer=l2(0.001), kernel_initializer='he_normal'),
    BatchNormalization(),
    #Dropout
    Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001), kernel_initializer='glorot_normal')

])
my_callback_es = tensorflow.keras.callbacks.EarlyStopping(patience = 5, restore_best_weights = True)
my_callback_rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=3, factor=0.5, min_lr=0.00001, verbose=1)
model.compile(optimizer = RMSprop(learning_rate=0.0005),loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
batchsize = 16
history = model.fit(train_datagen.flow_from_directory('train/', 
                                                      batch_size = batchsize,
                                                      target_size=(256, 256),
                                                     class_mode = 'binary'),
                    validation_data = valid_datagen.flow_from_directory('validation/',
                                                     batch_size = batchsize,
                                                     target_size = (256,256),
                                                     class_mode = 'binary'),
                    steps_per_epoch = len(train_fnames)/10/batchsize,
                    callbacks = [my_callback_es,my_callback_rlr],
                    epochs = 50
                   )

In [ ]:
plt.xlabel('epochs')
plt.ylabel('loss')
plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'validation_loss')
plt.legend()
plt.plot()